# Quantum Circuit to add any two numbers.

In [1]:
from qiskit import Aer, QuantumCircuit, QuantumRegister, ClassicalRegister, assemble
sim = Aer.get_backend('aer_simulator')

In [2]:
def numbers_to_ckt(x, y):
    xbin = bin(x).replace("0b", "", 1)
    ybin = bin(y).replace("0b", "", 1)
    #assert(len(xbin) == len(ybin))
    n = max(len(xbin), len(ybin)) # now we can add any two numbers irrespective of relative size.
    qbits_x = QuantumRegister(n, "first number")
    qbits_y = QuantumRegister(n, "second number")
    qbits_ans = QuantumRegister(n+1, "answer qubits")
    cbits_ans = ClassicalRegister(n+1, "answer cbits")
    c = QuantumCircuit(qbits_x, qbits_y, qbits_ans, cbits_ans)
    for i in range(len(xbin)): 
        if xbin[len(xbin)-1-i] == "1": c.x(i)
    for i in range(len(ybin)):
        if ybin[len(ybin)-1-i] == "1": c.x(n + i)
    c.barrier()
    return c, n

In [3]:
def carryFromAddnOf3bits(ckt, bit1, bit2, bit3, carrybit):
    """expects carrybit is |0> initially. The algorithm will then place its final value in it."""
    # idea: carrybt == 1 iff at least two of the three bits are 1. use ccx to check for the case of two bits being 1, and coincidentally(since 1 xor 1 xor 1 = 1 as well) the case of all being 1 is also taken care of.
    ckt.ccx(bit1, bit2, carrybit)
    ckt.ccx(bit2, bit3, carrybit)
    ckt.ccx(bit3, bit1, carrybit)
    ckt.barrier()

In [4]:
def add(x, y):
    ckt, n = numbers_to_ckt(x, y) # n is the number of bits in x and y
    
    for i in range(n):
        # get the carry of the ith digit addition onto the i+1th answer qubit. Of course the modulo 2^n is taken care by the if not statement. To add them fully(no truncating if ans exceeds 2^n), add one more answer bit and remove the if not. -- we'll do that, the mod n answer can be extracted to be the last n digits of the answer qubits.
        # if not i == n-1:
        #     carryFromAddnOf3bits(ckt, i, n+i, 2*n+i, 2*n + i + 1)
        carryFromAddnOf3bits(ckt, i, n+i, 2*n+i, 2*n+i+1)
        # now store the ith digit in qubit 2n+i
        ckt.cx(i, 2*n + i)
        ckt.cx(n + i, 2*n + i) # current digit = its prev value(carry)
        ckt.barrier()
        
    # finally measure the answer qubits.
    for i in range(n+1): 
        ckt.measure(2*n + i, i)
    return ckt


In [5]:
ckt = add(37, 63)
#ckt.draw("mpl")

In [6]:
answer = list(sim.run(assemble(ckt)).result().get_counts())[0]
print(int(answer, 2))

100
